In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale,PolynomialFeatures
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold,RFE
from minepy import MINE
from mlxtend.regressor import StackingRegressor

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [27]:
def evalerror(y, y_pred):
    loss = np.sum(np.square(y - y_pred))
    n = len(y)
    return loss / n

In [123]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('train/test.csv')
y = pd.read_csv('train/y.csv')

In [125]:
#GBDT筛选特征
clf_gt2 = ensemble.GradientBoostingRegressor(max_depth=1, n_estimators=320, random_state=1)
clf_gt2.fit(train, y)


model1 = SelectFromModel(clf_gt2, prefit=True) 
train = pd.DataFrame(model1.transform(train))
test = pd.DataFrame(model1.transform(test))

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [141]:
#单xgb模型线下5折 交叉验证
def wmaeEval(preds, dtrain):
    label = dtrain.get_label()
    return 'error', np.sum(np.square(preds - label)) / len(label)

param = {}
param['eta'] = 0.01
param['max_depth'] = 3
param['subsample'] = 0.8
param['colsample_bytree'] = 0.3

param['seed'] = 1
num_round = 10000

xgbTrain = xgb.DMatrix(train, label=y)
modle = xgb.cv(param, xgbTrain, num_boost_round=4200, feval=wmaeEval, nfold=5)
print(modle.iloc[-1, 0]) 

0.025204


In [ ]:
#单lgb模型线下5折 交叉验证
params = {}
params['learning_rate'] = 0.01
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'         
params['feature_fraction'] = 0.3      
params['bagging_fraction'] = 0.8 
params['num_leaves'] = 64  
result = []   
folds = list(KFold(len(train), n_folds=5, random_state=0))
for j, (train_idx, test_idx) in enumerate(folds):
    print("the folds", j)
    X_train = train.ix[train_idx, :]
    X_valid = train.ix[test_idx, :]
        
    Y_train = y.ix[train_idx, :]
    Y_valid = y.ix[test_idx, :]
    d_train = lgb.Dataset(X_train, label=Y_train['Y'])
    clf = lgb.train(params, d_train, 620)
    preds = clf.predict(X_valid)
    result.append(evalerror(preds, Y_valid['Y']))
    
np.mean(result)

In [37]:
#线下Stacking 取20%数据集进行验证
x_train, x_valid, y_train, y_valid = train_test_split(train, y, test_size = 0.2, random_state=1)
x_train.reset_index(inplace=True)
x_train.drop(['index'], axis=1, inplace=True)

y_train.reset_index(inplace=True)
y_train.drop(['index'], axis=1, inplace=True)

y_valid.reset_index(inplace=True)
y_valid.drop(['index'], axis=1, inplace=True)

c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\scarlet\appdata\local\programs\python\python35\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
model_gb = ensemble.GradientBoostingRegressor(n_estimators=450, 
                                              max_depth=2, 
                                              subsample=0.8, 
                                              learning_rate=0.01, 
                                              random_state=0, 
                                              max_features=0.2)
modle0  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.2, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=2100)
modle1  = xgb.XGBRegressor(learning_rate=0.01, 
                           max_depth=3, 
                           colsample_bytree=0.3, 
                           subsample=0.8, 
                           seed=0, 
                           n_estimators=1600,
                           min_child_weight=6)

clf1 = lgb.LGBMRegressor(colsample_bytree=0.3,
                         learning_rate=0.01, 
                         subsample=0.8, 
                         num_leaves=4, 
                         objective='regression', 
                         n_estimators=350, 
                         seed=0)
base_model = [['xgb0', modle0],
              ['xgb1', modle1], 
              ['gb', model_gb],
              ['lgb', clf1],]

folds = list(KFold(len(x_train), n_folds=5, random_state=0))
S_train = np.zeros((x_train.shape[0], len(base_model)))
S_test = np.zeros((x_valid.shape[0], len(base_model)))    
for index, item in enumerate(base_model):
    print("the model", index)
    clf = item[1]
    S_test_i = np.zeros((x_valid.shape[0], len(folds)))
    for j, (train_idx, test_idx) in enumerate(folds):
        X_train = x_train.ix[train_idx, :]
        X_valid = x_train.ix[test_idx, :]
        Y = y_train.ix[train_idx, :]
        clf.fit(X_train, Y['Y'])
        S_train[test_idx, index] = clf.predict(X_valid)
        S_test_i[:, j] = clf.predict(x_valid)       
    S_test[:, index] = S_test_i.mean(1)
    
linreg = linear_model.LinearRegression()
linreg.fit(S_train, y_train)

result = linreg.predict(S_test)
evalerror(pd.DataFrame(result)[0],  y_valid['Y'])

the model 0
the model 1
the model 2
the model 3


0.027016934367817883